# MAPD-B distributed processing exam
## Project 4: Streaming processing of cosmic rays using Drift Tubes detectors

The goal of this project is to reproduce a real-time processing of real data collected in a
particle physics detector and publish the results in a dashboard for live monitoring.

### Students:
* [Hilario Capettini](https://github.com/hcapettini2) (2013031)
* [Javier Gerardo Carmona](https://github.com/eigen-carmona/) (2005005)
* [Saverio Monaco](https://github.com/SaverioMonaco/) (2012264)

##  The Cluster

5 Cloud Veneto VMs were assigned for this project:
* MAPD-B_Gr17-5 10.67.22.83
* MAPD-B_Gr17-4 10.67.22.136
* MAPD-B_Gr17-3 10.67.22.102
* MAPD-B_Gr17-2 10.67.22.39
* MAPD-B_Gr17-1 10.67.22.137

Each machine runs CentOs and the Specs are:
* RAM:   8GB
* VCPUs: 4
* Disk:  25GB

## Setting the Cluster

In [ ]:
# Maybe here we can spend a few words on how to set up a Cluster

### Configuration chosen

### Benchmarks of other configurations

In [1]:
# Here we basically justify why we chosen the current configuration

## Setting Kafka and Spark

In [ ]:
# Maybe here we can spend a few words about the commands on how to create the pipeline,
# in a cluster

### Consumer and Producer

## Data Processing

In [13]:
flatDF.printSchema()

root
 |-- HEAD: integer (nullable = true)
 |-- FPGA: integer (nullable = true)
 |-- TDC_CHANNEL: integer (nullable = true)
 |-- ORBIT_CNT: double (nullable = true)
 |-- BX_COUNTER: integer (nullable = true)
 |-- TDC_MEAS: double (nullable = true)



In [ ]:
## Colection of functions for the main computation

def chamber_assignment(df):
    '''Assign chamber number and leave the scintillator carriers with chamber == null'''
    
    return(df.withColumn('CHAMBER',when(col("FPGA") == 0, 
                                                when(col("TDC_CHANNEL")<=63,1).\
                                                otherwise(when(col("TDC_CHANNEL")<128,2))).\
                                           otherwise(when(col("TDC_CHANNEL")<=63,3).\
                                                     otherwise(when(col("TDC_CHANNEL")<128,4))
                                           )).\
                                           select([ col('TDC_CHANNEL'), col('ORBIT_CNT'),
                                           col('BX_COUNTER'),col('TDC_MEAS'),
                                           col('CHAMBER')])
          )

def scintillator_data(df):
    '''Define a dataframe containing the relevant information for 
    the scintillator analysis''' 
    
    #First we filter the events encoding the passage time,
    #then we add the PASSAGE time for each event 
    #Finally if we have two scilantor hits within the same orbit we keep 
    #the one with the smaller time
    return(df.filter((col("CHAMBER").isNull())).\
                          withColumn("PASSAGETIME", 25 * (col("ORBIT_CNT") * 3564 + col("BX_COUNTER") + col("TDC_MEAS")/30)).\
                          drop("TDC_CHANNEL").drop("BX_COUNTER").\
                          drop("TDC_MEAS").drop("CHAMBER").\
                          groupBy("ORBIT_CNT").min("PASSAGETIME").\
                          withColumnRenamed("ORBIT_CNT","ORBIT_CNT_sci").\
                          withColumnRenamed("min(PASSAGETIME)","PASSAGETIME")
          )


def histogram_1(df,min_v_1,max_v_1,inc_1):
    '''This function return the bins and counts for the first requested histogram'''
    hist_1_bins = np.arange(min_v_1,max_v_1,inc_1)
    hist_1 = df\
        .filter((min_v_1<=F.col('TDC_CHANNEL')) & (F.col('TDC_CHANNEL')<=max_v_1))\
        .withColumn('BIN', F.floor((F.col('TDC_CHANNEL')-min_v_1)/inc_1))\
        .groupBy('CHAMBER','BIN')\
        .count().select('CHAMBER','BIN', col('count').alias('COUNT'))
    return (hist_1_bins, hist_1)
    

def histogram_2(df,min_v_2,max_v_2,inc_2):
    '''This function return the bins and counts for the second requested histogram'''
    hist_2_bins = np.arange(min_v_2,max_v_2,inc_2)
    hist_2 = df\
        .groupBy('CHAMBER','ORBIT_CNT')\
        .agg(F.countDistinct('TDC_CHANNEL').alias('ACTIVE_CHANNELS'))\
        .filter((min_v_2<=F.col('ORBIT_CNT'))&(F.col('ORBIT_CNT')<=max_v_2))\
        .withColumn('BIN',F.floor((F.col('ORBIT_CNT')-min_v_2)/inc_2))\
        .groupBy('CHAMBER','BIN')\
        .agg(F.sum('ACTIVE_CHANNELS').alias('COUNT'))#.collect()
    return(hist_2_bins, hist_2)

def histogram_3(df,min_v_3,max_v_3,inc_3):
    '''This function return the bins and counts for the third requested histogram'''
    hist_3_bins = np.arange(min_v_3,max_v_3,inc_3)
    hist_3 = df\
        .filter((min_v_3<=F.col('TDC_CHANNEL')) & (F.col('TDC_CHANNEL')<=max_v_3))\
        .withColumn('BIN', F.floor((F.col('TDC_CHANNEL')-min_v_3)/inc_3))\
        .groupBy('CHAMBER','BIN')\
        .count().select('CHAMBER','BIN', col('count').alias('COUNT')) 
    return(hist_3_bins, hist_3)

def histogram_4(df,min_v_4,max_v_4,inc_4):
    '''This function return the bins and counts for the fourth requested histogram'''
    hist_4_bins = np.arange(min_v_4,max_v_4,inc_4)
    hist_4 = df\
        .filter((min_v_4<=F.col('DRIFTIME')) & (F.col('DRIFTIME')<=max_v_4))\
        .withColumn('BIN', F.floor((F.col('DRIFTIME')-min_v_4)/inc_4))\
        .groupBy('CHAMBER','BIN')\
        .count().select('CHAMBER','BIN', col('count').alias('COUNT')) 
    return(hist_4_bins, hist_4)


   
def numpify(bins, pos_count):
    '''NUMPIFY RESULTS'''
    counter = np.zeros(len(bins))#np.zeros(len(bins)-1)?
    positions = np.array(list(pos_count.keys()))
    counts = np.array(list(pos_count.values()))
    counter[positions] = counts
    return counter


def prepare_results(chamber_hits,hist_1,hist_2,hist_3,hist_4,hist_1_bins,hist_2_bins,hist_3_bins,hist_4_bins):
    '''COLLECTING RESULTS'''
    _chamber_hits = chamber_hits.collect()
    
    _hist_1 = hist_1.groupBy('CHAMBER').agg(
    F.map_from_entries(
        F.collect_list(
            F.struct("BIN", "COUNT"))).alias("COUNT")
        ).collect()

    _hist_2 = hist_2.groupBy('CHAMBER').agg(
    F.map_from_entries(
        F.collect_list(
            F.struct("BIN","COUNT"))).alias("COUNT")
        ).collect()

    _hist_3 = hist_3.groupBy('CHAMBER').agg(
    F.map_from_entries(
        F.collect_list(
            F.struct("BIN", "COUNT"))).alias("COUNT")
        ).collect()
    
    _hist_4 = hist_4.groupBy('CHAMBER').agg(
    F.map_from_entries(
        F.collect_list(
            F.struct("BIN", "COUNT"))).alias("COUNT")
        ).collect()
    

    # JSON FORMATING OF RESULTS
    _hist_1_dict = {row.CHAMBER: {
        'Bins': list(hist_1_bins), 'Counts': list(numpify(hist_1_bins,row.COUNT))
    } for row in _hist_1}

    _hist_2_dict = {row.CHAMBER: {
        'Bins': list(hist_2_bins), 'Counts': list(numpify(hist_2_bins,row.COUNT))
    } for row in _hist_2}
    
    _hist_3_dict = {row.CHAMBER: {
        'Bins': list(hist_3_bins), 'Counts': list(numpify(hist_3_bins,row.COUNT))
    } for row in _hist_3}
        
    _hist_4_dict = {row.CHAMBER: {
        'Bins': list(hist_4_bins), 'Counts': list(numpify(hist_4_bins,row.COUNT))
    } for row in _hist_4}
    
    return(_chamber_hits,_hist_1_dict,_hist_2_dict,_hist_3_dict,_hist_4_dict)

In [ ]:
def computations(df, epoch):
    '''This is the main function of the code, it requires a dataframe as input. The dataframe is analysed
       and the results are published in the kafka topic "results" '''
    
    #start=time.time()
    main_df = chamber_assignment(df)

    scintillator_df = scintillator_data(main_df)
    
    #Drop the columns with null values from main_df
    hit_df = main_df.na.drop(subset=["CHAMBER"])#.show()
    
    ## TOTAL NUMBER OF PROCESSED HITS
    total_hits = hit_df.count()
    if not total_hits: return

    ## TOTAL NUMBER OF PROCESSED HITS PER CHAMBER
    chamber_hits = hit_df\
        .groupBy('CHAMBER').count()\
        .select(col('CHAMBER'),col('count').alias('COUNT'))#.collect()
    
    ## ACTIVE TDC_CHANNEL PER CHAMBER
    min_v_1 = 0
    max_v_1 = 170
    inc_1 = 5
    hist_1_bins, hist_1 = histogram_1(hit_df,min_v_1,max_v_1,inc_1)
    
    ## ACTIVE TDC_CHANNEL PER CHAMBER PER ORBIT_CNT
    min_v_2 = 6.e5 #main_df.agg(F.min(F.col('ORBIT_CNT')).alias('min')).collect()[-1].min
    max_v_2 = 1.e7 #main_df.agg(F.max(F.col('ORBIT_CNT')).alias('max')).collect()[-1].max
    inc_2 = 0.5e6
    hist_2_bins, hist_2 = histogram_2(hit_df,min_v_2,max_v_2,inc_2)
    
    
    #keep only the hits with a scintillator signal within the same orbit
    chamber_sci = hit_df.join(scintillator_df,main_df.ORBIT_CNT ==  scintillator_df.ORBIT_CNT_sci,"inner")

    ## ADD TIME CORRECTION BY CHAMBER
    chamber_sci = chamber_sci.withColumn('TIME_OFFSET',when(col("CHAMBER") == 1, 93.9).\
                                                       when(col("CHAMBER") == 2, 101.4).\
                                                       when(col("CHAMBER") == 3, 95.5).\
                                                       when(col("CHAMBER") == 4, 92.4))

    #Add the ABSSOLUTETIME and DRIFTIME
    chamber_sci = chamber_sci.withColumn("ABSOLUTETIME",
                             25 * (col("ORBIT_CNT") * 3564 + col("BX_COUNTER") + col("TDC_MEAS")/30)).\
                              withColumn("DRIFTIME",col("ABSOLUTETIME")-col("PASSAGETIME") + col("TIME_OFFSET"))
   

    ## ACTIVE TDC_CHANNEL PER CHAMBER WITHIN SCINTILLATOR SIGNAL
    min_v_3 = 0
    max_v_3= 170
    inc_3 = 5
    hist_3_bins, hist_3 = histogram_3(chamber_sci,min_v_3,max_v_3,inc_3)
    

    ## HISTOGRAM OF DRIFTIME, PER CHAMBER
    min_v_4 = 0
    max_v_4= 1000
    inc_4 = 10
    hist_4_bins, hist_4 = histogram_4(chamber_sci,min_v_4,max_v_4,inc_4)
    
    #PREPARE THE RESULTS
    _chamber_hits,_hist_1_dict,_hist_2_dict,_hist_3_dict,_hist_4_dict = prepare_results(chamber_hits,hist_1,hist_2,hist_3,hist_4,hist_1_bins,hist_2_bins,hist_3_bins,hist_4_bins)
    
    
    results = {f'Chamber_{row.CHAMBER}': {
        'Count': int(row.COUNT),
        'Hist_1': _hist_1_dict.get(row.CHAMBER, {'Bins': list(np.arange(min_v_1,max_v_1,inc_1)), 'Counts' : [0]*(len(list(np.arange(min_v_1,max_v_1,inc_1)))-1)}),
        'Hist_2': _hist_2_dict.get(row.CHAMBER, {'Bins': list(np.arange(min_v_2,max_v_2,inc_2)), 'Counts' : [0]*(len(list(np.arange(min_v_2,max_v_2,inc_2)))-1)}),
        'Hist_3': _hist_3_dict.get(row.CHAMBER, {'Bins': list(np.arange(min_v_3,max_v_3,inc_3)), 'Counts' : [0]*(len(list(np.arange(min_v_3,max_v_3,inc_3)))-1)}),
        'Hist_4': _hist_4_dict.get(row.CHAMBER, {'Bins': list(np.arange(min_v_4,max_v_4,inc_4)), 'Counts' : [0]*(len(list(np.arange(min_v_4,max_v_4,inc_4)))-1)})} for row in _chamber_hits}

    results.update({
        'Index': time.time(),
        'Total Count': int(total_hits)
    })

    producer.send(topic="results", value= str(results).encode('utf-8'))
    #producer.flush()
    end = time.time()

    print("Time =",end-start)

##  Results

### Vertical scalability

### Horizontal scalability

### Scaling with ammount of data

## Live Plotting

To create a live webpage dashboard we used [Plotly Dash](https://github.com/plotly/dash) a Python library built on top of Plotly to create Analytical Web Apps.

The information reported in the Dashboard are the following:

**PLOTS**
1. total number of processed hits, post-clensing (PLOT AND TABLE)
2. total number of processed hits, post-clensing, per chamber (TABLE)
3. histogram of the counts of active TDC_CHANNEL, per chamber (HISTOGRAM 1)
4. histogram of the total number of active TDC_CHANNEL in each ORBIT_CNT, per chamber (HISTOGRAM 2)

**EXTRA**
1. histogram of the counts of active TDC_CHANNEL, per chamber, ONLY for those orbits with at least one scintillator signal in it (EXTRA 1)
2. histogram of the DRIFTIME, per chamber (EXTRA 2 AND EXTRA 2 (cumulative))

<img src="./imgs/dashboard.png"/>

## Backup Information

### Data-cleansing

Data-cleansing : $$\text{HEAD} == 2 $$
Other entries provide ancillary information

### Chamber mapping

• Chamber 0 → (FPGA = 0) AND (TDC_CHANNEL in [0-63])\
• Chamber 1 → (FPGA = 0) AND (TDC_CHANNEL in [64-127])\
• Chamber 2 → (FPGA = 1) AND (TDC_CHANNEL in [0-63])\
• Chamber 3 → (FPGA = 1) AND (TDC_CHANNEL in [64-127])

### Driftime

#### Absolute time
For each hit we can associate an absolute time:

$$t_{TDC\space hit} = 25 ∗ ( ORBIT\_CNT ∗ 3564 + BX\_COUNTER + TDC\_MEAS /30)\quad [ns]$$


#### Passage of a muon time
The passage time of any muon is provided by an external scintillator signal which correspond to the following selection:

$$\text{(FPGA == 1) AND (TDC_CHANNEL == 128)}$$

#### Scintillator time offset

In [2]:
# scintillator time offset by Chamber
time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}

#### Driftime
For those hits with a scintillator signal within the same orbit, a DRIFTIME can be defined, corresponding to the ABSOLUTETIME difference between each hit and the scintillator (from the same orbit).